# EXPLORE Telecommunication CDRs Data of Trentino & Milano Italy From Harvard Dataverse

cc : [Naufal Nashif](https://www.linkedin.com/in/naufalnashif/)
| files : [Github](https://github.com/naufalnashif/test-case/tree/c7834a26f2cb0bf193fa1a37154b644f4ac432c4/telkomsel)

# Import Dependencies

In [ ]:
import requests
from google.colab import userdata
import pandas as pd
from IPython.display import display
import os
import glob

# Init

In [ ]:
dataset_trentino_url = "https://dataverse.harvard.edu/api/datasets/:persistentId?persistentId=doi:10.7910/DVN/QLCABU"
dataset_milano_url = "https://dataverse.harvard.edu/api/datasets/:persistentId?persistentId=doi:10.7910/DVN/EGZHFV"
grid_trentino_url = "https://dataverse.harvard.edu/api/datasets/:persistentId?persistentId=doi:10.7910/DVN/FZRVSX"
grid_milano_url = "https://dataverse.harvard.edu/api/datasets/:persistentId?persistentId=doi:10.7910/DVN/QJWLFU"
api_token = userdata.get("HARVARD_API_TOKEN")
folder_trentino = "raw_data_trentino"
folder_milano = "raw_data_milano"

In [ ]:
class Harvard_Dataset_Download:
    def __init__(self, dataset_url, api_token):
        self.dataset_url = dataset_url
        self.api_token = api_token

    def get_available_files(self):

        headers = {
            'X-Dataverse-key': self.api_token
        }
        response = requests.get(self.dataset_url, headers=headers)

        if response.status_code == 200:

            data = response.json()
            files = data['data']['latestVersion']['files']

            file_info = []

            for file in files:
                file_label = file['label']
                datafile_id = file['dataFile']['id']
                file_info.append({
                    'nama_file': file_label,
                    'datafile_id': datafile_id
                })

            return pd.DataFrame(file_info)
        else:
            print(f"Terjadi kesalahan saat mengambil metadata dataset: {response.status_code}, {response.text}")
            return None

    def download_files(self, folder_name, num_files=30):

        df_files = self.get_available_files()

        if df_files is not None:

            df_files_to_download = df_files.head(num_files)

            if not os.path.exists(folder_name):
                os.makedirs(folder_name)

            headers = {
                'X-Dataverse-key': self.api_token
            }

            for index, row in df_files_to_download.iterrows():
                file_name = row['nama_file']
                datafile_id = row['datafile_id']

                print(f"Menemukan file: {file_name} dengan datafile_id: {datafile_id}")


                file_url = f"https://dataverse.harvard.edu/api/access/datafile/{datafile_id}"


                file_response = requests.get(file_url, headers=headers)
                if file_response.status_code == 200:

                    file_path = os.path.join(folder_name, file_name)
                    with open(file_path, 'wb') as f:
                        f.write(file_response.content)
                    print(f"File {file_name} berhasil diunduh ke folder {folder_name}.")
                else:
                    print(f"Terjadi kesalahan saat mengunduh file {file_name}: {file_response.status_code}")
        else:
            print("Tidak ada file untuk diunduh.")



In [ ]:
class DataWrangling:
    def __init__(self, folder_name, col_list):
        self.folder_name = folder_name
        self.col_list = col_list

    def trentino_wrangling (self):
        #Initialize an empty dataframe to append daily and hourly resampled data
        dailyGridActivityTrentino = pd.DataFrame()
        hourlyGridActivityTrentino = pd.DataFrame()

        #Create a list of 30 data file names placed under directory  "./{folder_name}}/" with extension .txt
        filenames = glob.glob(f"./{self.folder_name}/*.txt")

        #Set the column names for the data read
        col_list = self.col_list

        #Read each tab separated file into Pandas DataFrame
        for file in filenames:
            read_data = pd.read_csv(file, sep='\t',header=None, names=col_list, parse_dates=True)

            #Convert timeInterval column which has Epoch timestamps to UTC and then convert to Milan's local timezone and save it to column startTime
            #tz_localize(None) returns local time format instead of "UTC+1:00" format
            read_data['startTime'] = pd.to_datetime(read_data.timeInterval, unit='ms', utc=True).dt.tz_convert('CET').dt.tz_localize(None)

            #Drop timeInterval & countryCode columns
            read_data.drop(columns=['timeInterval','countryCode'], inplace=True)

            #Groupby gridID and startTime, startTime which is 10 min apart is resampled to daily aggregation
            read_data_daily = read_data.groupby(['gridID', pd.Grouper(key='startTime', freq='D')]).sum()
            dailyGridActivityTrentino = pd.concat([dailyGridActivityTrentino,read_data_daily]).groupby(['gridID', 'startTime']).sum()

            #Groupby gridID and startTime, startTime which is 10 min apart is resampled to hourly aggregation
            read_data_hourly = read_data.groupby(['gridID', pd.Grouper(key='startTime', freq='h')]).sum()
            hourlyGridActivityTrentino = pd.concat([hourlyGridActivityTrentino,read_data_hourly]).groupby(['gridID', 'startTime']).sum()

        #Get Grid wise total volume of the activities over the months
        totalGridActivityTrentino = dailyGridActivityTrentino.groupby('gridID').sum()
        return dailyGridActivityTrentino, hourlyGridActivityTrentino, totalGridActivityTrentino

    def milano_wrangling (self):
        #Initialize an empty dataframe to append daily and hourly resampled data
        dailyGridActivityMilano = pd.DataFrame()
        hourlyGridActivityMilano = pd.DataFrame()

        #Create a list of 30 data file names placed under directory  "./raw_data_Milano/" with extension .txt
        filenames = glob.glob(f"./{self.folder_name}/*.txt")

        #Set the column names for the data read
        col_list = self.col_list

        #Read each tab separated file into Pandas DataFrame
        for file in filenames:
            read_data = pd.read_csv(file, sep='\t',header=None, names=col_list, parse_dates=True)

            #Convert timeInterval column which has Epoch timestamps to UTC and then convert to Milan's local timezone and save it to column startTime
            #tz_localize(None) returns local time format instead of "UTC+1:00" format
            read_data['startTime'] = pd.to_datetime(read_data.timeInterval, unit='ms', utc=True).dt.tz_convert('CET').dt.tz_localize(None)

            #Drop timeInterval & countryCode columns
            read_data.drop(columns=['timeInterval','countryCode'], inplace=True)

            #Groupby gridID and startTime, startTime which is 10 min apart is resampled to daily aggregation
            read_data_daily = read_data.groupby(['gridID', pd.Grouper(key='startTime', freq='D')]).sum()
            dailyGridActivityMilano = pd.concat([dailyGridActivityMilano,read_data_daily]).groupby(['gridID', 'startTime']).sum()

            #Groupby gridID and startTime, startTime which is 10 min apart is resampled to hourly aggregation
            read_data_hourly = read_data.groupby(['gridID', pd.Grouper(key='startTime', freq='h')]).sum()
            hourlyGridActivityMilano = pd.concat([hourlyGridActivityMilano,read_data_hourly]).groupby(['gridID', 'startTime']).sum()

        #Get Grid wise total volume of the activities over the months
        totalGridActivityMilano = dailyGridActivityMilano.groupby('gridID').sum()

        return dailyGridActivityMilano, hourlyGridActivityMilano, totalGridActivityMilano

# Get The Data

In [ ]:
# Trentino
dataset_downloader = Harvard_Dataset_Download(dataset_trentino_url, api_token)
dataset_downloader.download_files(folder_trentino, num_files=30)

print("------------------------------------------------")
# Milano
dataset_downloader = Harvard_Dataset_Download(dataset_milano_url, api_token)
dataset_downloader.download_files(folder_milano, num_files=30)

Menemukan file: sms-call-internet-tn-2013-11-01.txt dengan datafile_id: 2674712
File sms-call-internet-tn-2013-11-01.txt berhasil diunduh ke folder raw_data_trentino.
Menemukan file: sms-call-internet-tn-2013-11-02.txt dengan datafile_id: 2674718
File sms-call-internet-tn-2013-11-02.txt berhasil diunduh ke folder raw_data_trentino.
Menemukan file: sms-call-internet-tn-2013-11-03.txt dengan datafile_id: 2674726
File sms-call-internet-tn-2013-11-03.txt berhasil diunduh ke folder raw_data_trentino.
Menemukan file: sms-call-internet-tn-2013-11-04.txt dengan datafile_id: 2674730
File sms-call-internet-tn-2013-11-04.txt berhasil diunduh ke folder raw_data_trentino.
Menemukan file: sms-call-internet-tn-2013-11-05.txt dengan datafile_id: 2674734
File sms-call-internet-tn-2013-11-05.txt berhasil diunduh ke folder raw_data_trentino.
Menemukan file: sms-call-internet-tn-2013-11-06.txt dengan datafile_id: 2674738
File sms-call-internet-tn-2013-11-06.txt berhasil diunduh ke folder raw_data_trentino

In [ ]:
# Trentino
grid_trentino_downloader = Harvard_Dataset_Download(grid_trentino_url, api_token)
grid_trentino_downloader.get_available_files()

,nama_file,datafile_id
0,trentino-grid.geojson,2674653


In [ ]:
# Milano
grid_milano_downloader = Harvard_Dataset_Download(grid_milano_url, api_token)
grid_milano_downloader.get_available_files()

,nama_file,datafile_id
0,milano-grid.geojson,2670806


In [ ]:
# Trentino
grid_trentino_downloader.download_files(folder_trentino ,num_files=1)

Menemukan file: trentino-grid.geojson dengan datafile_id: 2674653
File trentino-grid.geojson berhasil diunduh ke folder raw_data_trentino.


In [ ]:
import geopandas as gpd

# Baca file GeoJSON
trentino = gpd.read_file('./raw_data_trentino/trentino-grid.geojson')

# Ekstrak koordinat dari geometri
trentino['geometry_wkt'] = trentino['geometry'].apply(lambda x: x.wkt)


# Simpan ke CSV
trentino[['cellId', 'geometry_wkt']].to_csv('trentino_grid.csv', index=False)


In [ ]:
# Milano
grid_milano_downloader.download_files(folder_milano, num_files=1)

Menemukan file: milano-grid.geojson dengan datafile_id: 2670806
File milano-grid.geojson berhasil diunduh ke folder raw_data_milano.


In [ ]:
# Baca file GeoJSON
milano = gpd.read_file('./raw_data_milano/milano-grid.geojson')

# Ekstrak koordinat dari geometri
milano['geometry_wkt'] = milano['geometry'].apply(lambda x: x.wkt)


# Simpan ke CSV
milano[['cellId', 'geometry_wkt']].to_csv('milano_grid.csv', index=False)


# Data Wrangling

In [ ]:
col_list = ['gridID', 'timeInterval', 'countryCode', 'smsIn', 'smsOut', 'callIn', 'callOut', 'internet']

In [ ]:
# Trentino
trentino_data_wrangler = DataWrangling(folder_trentino, col_list)
dailyGridActivityTrentino, hourlyGridActivityTrentino, totalGridActivityTrentino = trentino_data_wrangler.trentino_wrangling()

# Milano
milano_data_wrangler = DataWrangling(folder_milano, col_list)
dailyGridActivityMilano, hourlyGridActivityMilano, totalGridActivityMilano = milano_data_wrangler.milano_wrangling()

In [ ]:
dailyGridActivityMilano.head()

smsIn     smsOut     callIn    callOut     internet  \
gridID startTime                                                              
1      2013-11-01   78.709755  45.886570  41.108567  48.245378  1507.048349   
       2013-11-02   86.415810  43.875946  47.891016  53.590637  1515.641856   
       2013-11-03   77.728292  45.446780  36.145436  40.906425  1533.148425   
       2013-11-04  104.793806  54.821018  67.898464  70.399418  1404.813593   
       2013-11-05   97.425105  46.607029  68.735213  70.766221  1518.090111   

                    tipe    city  
gridID startTime                  
1      2013-11-01  Daily  Milano  
       2013-11-02  Daily  Milano  
       2013-11-03  Daily  Milano  
       2013-11-04  Daily  Milano  
       2013-11-05  Daily  Milano

In [ ]:
# Reset multiindex untuk semua DataFrame
dailyGridActivityTrentino = dailyGridActivityTrentino.reset_index()
hourlyGridActivityTrentino = hourlyGridActivityTrentino.reset_index()
dailyGridActivityMilano = dailyGridActivityMilano.reset_index()
hourlyGridActivityMilano = hourlyGridActivityMilano.reset_index()

# Tambahkan kolom tipe dan city untuk setiap DataFrame
dailyGridActivityTrentino['tipe'] = 'Daily'
dailyGridActivityTrentino['city'] = 'Trentino'

hourlyGridActivityTrentino['tipe'] = 'Hourly'
hourlyGridActivityTrentino['city'] = 'Trentino'

dailyGridActivityMilano['tipe'] = 'Daily'
dailyGridActivityMilano['city'] = 'Milano'

hourlyGridActivityMilano['tipe'] = 'Hourly'
hourlyGridActivityMilano['city'] = 'Milano'

# Gabungkan semua DataFrame dengan union (concat di pandas)
datamart_final = pd.concat([
    dailyGridActivityTrentino,
    hourlyGridActivityTrentino,
    dailyGridActivityMilano,
    hourlyGridActivityMilano
], ignore_index=True)

# Simpan ke CSV
datamart_final.to_csv('datamart_final.csv', index=False)

In [ ]:
# Tambahkan kolom city untuk membedakan Trentino dan Milano
milano['city'] = 'Milano'
trentino['city'] = 'Trentino'

# Lakukan join berdasarkan city dan cellId/gridID
datamart_milano = pd.merge(
    datamart_final[datamart_final['city'] == 'Milano'],
    milano,
    how='left',
    left_on='gridID',
    right_on='cellId'
)

datamart_trentino = pd.merge(
    datamart_final[datamart_final['city'] == 'Trentino'],
    trentino,
    how='left',
    left_on='gridID',
    right_on='cellId'
)

# Gabungkan hasil join dari Milano dan Trentino
datamart_with_grid = pd.concat([datamart_milano, datamart_trentino], ignore_index=True)

# Simpan ke file CSV
datamart_with_grid.to_csv('datamart_with_grid.csv', index=False)